# СЕМИНАР 3: МЕТОД МАКСИМАЛЬНОГО ПРАВДОПОДОБИЯ

На лекции мы рассмотрели теоретические основы метода максимального правдоподобия и даже вывели несколько оценок ММП руками.

Попробуем провернуть аналогичные операции в Python


## Практический минимум

In [76]:
import datetime
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots
from scipy import stats
from scipy.optimize import minimize

* Попробуем оценить параметры нормального распределения для следующей выборки:

In [ ]:
x = [3.2, 7.9, 5.4, 4.9, 6.2, 4.3]

1. Напишем ф-ю логарифма макс. правдоподобия, которая принимает на вход массив из двух параметров норм. распр-я и нашу выборку.

   Возвращает значение лог-функции правдоподобия

In [ ]:
def lnL(theta, x):
    mu = theta[0]
    s2 = np.exp(theta[1])

    x = np.array(x)
    n = x.size

    l = -0.5 * n * np.log(s2) - 0.5 / s2 * np.sum(
        (x - mu) ** 2
    )  # лог ф-я правдоподобия для норм. распр-я

    return (
        -l
    )  # на минус 1 домножаем в силу того, что в scipy есть только функция minimize


# Проверка, что работает
print(lnL([4, 0.2], x))

10.240554617493236


2. Оценим параметры

In [ ]:
theta_int = [0, 0]  # initial guess
res = minimize(lnL, theta_int, args=x)
res

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 5.353606615954048
        x: [ 5.317e+00  7.845e-01]
      nit: 14
      jac: [-1.788e-06  2.146e-06]
 hess_inv: [[ 3.527e-01  1.303e-02]
            [ 1.303e-02  3.198e-01]]
     nfev: 51
     njev: 17

In [6]:
mu, s2 = res.x
s2 = np.sqrt(np.exp(s2))
mu, s2

(np.float64(5.316665987334323), np.float64(1.480334570143922))

3. Получим тот же результат, используя встроенный метод stats.norm.fit(x)

In [7]:
mu, s = stats.norm.fit(x)
mu, s

(np.float64(5.316666666666666), np.float64(1.4803340463857775))

4. Построим 95% доверительный интервал для среднего

In [8]:
res.hess_inv

array([[0.35270628, 0.0130339 ],
       [0.0130339 , 0.31983887]])

In [ ]:
alpha = 0.05

z = stats.norm().ppf(1 - alpha / 2)

left = res.x[0] - z * np.sqrt(res.hess_inv[0, 0])
right = res.x[0] + z * np.sqrt(res.hess_inv[0, 0])

print(f"Доверительный интервал для среднего: [{left} ; {right}]")

Доверительный интервал для среднего: [4.152661404574153 ; 6.480670570094493]


Ниже на примере нормального распределения написан пайплайн запуска теста отношения правдоподобий

In [ ]:
np.random.seed(42)

n = 100  # размер выборки
mu_true = 0  # истинное среднее
sigma = 1  # известное стандартное отклонение
alpha = 0.05  # уровень значимости

hypo_mu = 0.15  # наше предположение о параметре как исследователей

# Генерируем выборку из нормального распределения
data = np.random.normal(loc=mu_true, scale=sigma, size=n)

# Оценка параметра при H1 (альтернативная гипотеза)
# MLE для параметра `mu` при нормальном распределении с известной дисперсией – выборочное среднее
mu_hat = np.mean(data)
print("Оцененное среднее:", mu_hat)

# Вычисление логарифмов правдоподобия
logL_R = np.sum(
    stats.norm.logpdf(data, loc=hypo_mu, scale=sigma)
)  # при mu = hypo_mu (H0)
logL_UR = np.sum(
    stats.norm.logpdf(data, loc=mu_hat, scale=sigma)
)  # при mu = mu_hat (H1)

# Вычисление статистики теста: -2 * (logL_R - logL_UR)
test_statistic = -2 * (logL_R - logL_UR)
print("Статистика теста: {:.4f}".format(test_statistic))

# Вычисление критического значения для уровня значимости alpha = 0.05
critical_value = stats.chi2.ppf(1 - alpha, df=1)
print("Критическое значение: {:.4f}".format(critical_value))

# Вычисление p-value c 1 степенью свободы, ибо мы ограничили один параметр
p_value = 1 - stats.chi2.cdf(test_statistic, df=1)
print("p-value: {:.4f}".format(p_value))

# Принятие решения
if p_value < alpha:
    print(f"Отвергаем H0: данные не согласуются с предположением mu = {hypo_mu}")
else:
    print(
        f"Нет оснований отвергать H0: данные согласуются с предположением mu = {hypo_mu}"
    )

Оцененное среднее: -0.10384651739409384
Статистика теста: 6.4438
Критическое значение: 3.8415
p-value: 0.0111
Отвергаем H0: данные не согласуются с предположением mu = 0.15


5. Попробуйте провести тест отношения правдоподобий в предыдущей задаче с гипотезой о том, что среднее равняется единице

In [ ]:
np.random.seed(42)
n = 100
mu_true = 0
sigma = 1
alpha = 0.05
hypo_mu = 1

data = np.random.normal(loc=mu_true, scale=sigma, size=n)
mu_hat = np.mean(data)

print("Оцененное среднее:", mu_hat)

# Вычисление логарифмов правдоподобия
logL_R = np.sum(stats.norm.logpdf(data, loc=hypo_mu, scale=sigma))
logL_UR = np.sum(stats.norm.logpdf(data, loc=mu_hat, scale=sigma))

# Вычисление статистики теста
test_statistic = -2 * (logL_R - logL_UR)
critical_value = stats.chi2.ppf(1 - alpha, df=1)
p_value = 1 - stats.chi2.cdf(test_statistic, df=1)

print("Статистика теста: {:.4f}".format(test_statistic))
print("Критическое значение: {:.4f}".format(critical_value))
print("p-value: {:.4f}".format(p_value))

fig1 = go.Figure()

fig1.add_trace(
    go.Histogram(
        x=data, name="Данные", nbinsx=20, opacity=0.7, histnorm="probability density"
    )
)

x_range = np.linspace(-4, 4, 200)

fig1.add_trace(
    go.Scatter(
        x=x_range,
        y=stats.norm.pdf(x_range, hypo_mu, sigma),
        mode="lines",
        name=f"Нулевая гипотеза N({hypo_mu}, {sigma}^2)",
        line=dict(color="red", width=2, dash="dash"),
    )
)

fig1.add_trace(
    go.Scatter(
        x=x_range,
        y=stats.norm.pdf(x_range, mu_hat, sigma),
        mode="lines",
        name=f"Альтернативная гипотеза N({mu_hat:.3f}, {sigma}^2)",
        line=dict(color="blue", width=2, dash="dot"),
    )
)

fig1.update_layout(
    title="Распределение данных и проверяемые гипотезы",
    xaxis_title="Значение",
    yaxis_title="Плотность вероятности",
)
fig1.show()

x_chi2 = np.linspace(0, 10, 300)
y_chi2 = stats.chi2.pdf(x_chi2, df=1)

fig2 = go.Figure()

fig2.add_trace(
    go.Scatter(
        x=x_chi2,
        y=y_chi2,
        mode="lines",
        name="chi^2 распределение (df=1)",
        line=dict(color="black", width=2),
        fill="tozeroy",
        fillcolor="rgba(0,100,80,0.1)",
    )
)

x_critical = np.linspace(critical_value, 10, 100)
y_critical = stats.chi2.pdf(x_critical, df=1)

fig2.add_trace(
    go.Scatter(
        x=x_critical,
        y=y_critical,
        mode="lines",
        name="Критическая область",
        line=dict(color="red", width=2),
        fill="tozeroy",
        fillcolor="rgba(255,0,0,0.3)",
    )
)

fig2.add_vline(
    x=test_statistic,
    line_dash="dash",
    line_color="blue",
    annotation_text=f"Статистика теста = {test_statistic:.3f}",
    annotation_position="top right",
)

fig2.add_vline(
    x=critical_value,
    line_dash="dash",
    line_color="green",
    annotation_text=f"Критическое значение = {critical_value:.3f}",
    annotation_position="top left",
)

fig2.update_layout(
    title="Распределение хи-квадрат и результат теста",
    xaxis_title="Значение статистики",
    yaxis_title="Плотность вероятности",
    showlegend=True,
    xaxis=dict(range=[0, 10]),
)
fig2.show()


if p_value < alpha:
    print(f"Отвергаем H0: данные не согласуются с предположением mu = {hypo_mu}")
else:
    print(
        f"Нет оснований отвергать H0: данные согласуются с предположением mu = {hypo_mu}"
    )

Оцененное среднее: -0.10384651739409384
Статистика теста: 121.8477
Критическое значение: 3.8415
p-value: 0.0000


Отвергаем H0: данные не согласуются с предположением mu = 1


6. А теперь тест отношения правдоподобий с гипотезой о том, что среднее равно единице, а дисперсия двойке

In [33]:
np.random.seed(42)
n = 100
mu_true = 0
sigma_true = 1
alpha = 0.05
hypo_mu = 1.0
hypo_sigma = np.sqrt(2.0)

data = np.random.normal(loc=mu_true, scale=sigma_true, size=n)

mu_hat = np.mean(data)
sigma_hat = np.std(data, ddof=0)

print("Оцененное среднее:", mu_hat)
print("Оцененное стандартное отклонение:", sigma_hat)
print("Оцененная дисперсия:", sigma_hat**2)

logL_R = np.sum(stats.norm.logpdf(data, loc=hypo_mu, scale=hypo_sigma))  # при H0
logL_UR = np.sum(stats.norm.logpdf(data, loc=mu_hat, scale=sigma_hat))  # при H1

test_statistic = -2 * (logL_R - logL_UR)
critical_value = stats.chi2.ppf(1 - alpha, df=2)
p_value = 1 - stats.chi2.cdf(test_statistic, df=2)

print("\nРезультаты теста:")
print("Статистика теста: {:.4f}".format(test_statistic))
print("Критическое значение (alpha=0.05, df=2): {:.4f}".format(critical_value))
print("p-value: {:.4f}".format(p_value))

fig1 = go.Figure()

fig1.add_trace(
    go.Histogram(
        x=data, name="Данные", nbinsx=20, opacity=0.7, histnorm="probability density"
    )
)

x_range = np.linspace(-4, 4, 200)

fig1.add_trace(
    go.Scatter(
        x=x_range,
        y=stats.norm.pdf(x_range, hypo_mu, hypo_sigma),
        mode="lines",
        name=f"Нулевая гипотеза N({hypo_mu}, {hypo_sigma**2:.1f})",
        line=dict(color="red", width=3, dash="dash"),
    )
)

fig1.add_trace(
    go.Scatter(
        x=x_range,
        y=stats.norm.pdf(x_range, mu_hat, sigma_hat),
        mode="lines",
        name=f"Альтернативная гипотеза N({mu_hat:.3f}, {sigma_hat**2:.3f})",
        line=dict(color="blue", width=3, dash="dot"),
    )
)

fig1.update_layout(
    title="Распределение данных и проверяемые гипотезы",
    xaxis_title="Значение",
    yaxis_title="Плотность вероятности",
    width=800,
    height=500,
)
fig1.show()

x_chi2 = np.linspace(0, 15, 300)
y_chi2 = stats.chi2.pdf(x_chi2, df=2)

fig2 = go.Figure()

fig2.add_trace(
    go.Scatter(
        x=x_chi2,
        y=y_chi2,
        mode="lines",
        name="chi^2 распределение (df=2)",
        line=dict(color="black", width=2),
        fill="tozeroy",
        fillcolor="rgba(0,100,80,0.1)",
    )
)

x_critical = np.linspace(critical_value, 15, 100)
y_critical = stats.chi2.pdf(x_critical, df=2)

fig2.add_trace(
    go.Scatter(
        x=x_critical,
        y=y_critical,
        mode="lines",
        name="Критическая область",
        line=dict(color="red", width=2),
        fill="tozeroy",
        fillcolor="rgba(255,0,0,0.3)",
    )
)

# Статистика теста
fig2.add_vline(
    x=test_statistic,
    line_dash="dash",
    line_color="blue",
    annotation_text=f"Статистика теста = {test_statistic:.3f}",
    annotation_position="top right",
)

fig2.add_vline(
    x=critical_value,
    line_dash="dash",
    line_color="green",
    annotation_text=f"Критическое значение = {critical_value:.3f}",
    annotation_position="top left",
)

fig2.update_layout(
    title="Распределение хи-квадрат и результат теста (df=2)",
    xaxis_title="Значение статистики",
    yaxis_title="Плотность вероятности",
    showlegend=True,
    xaxis=dict(range=[0, 15]),
    width=800,
    height=500,
)
fig2.show()


if p_value < alpha:
    print(
        f"\nОтвергаем H0: данные не согласуются с предположением μ = {hypo_mu}, sigma^2 = {hypo_sigma**2}"
    )
else:
    print(
        f"\nНет оснований отвергать H0: данные согласуются с предположением μ = {hypo_mu}, sigma^2 = {hypo_sigma**2}"
    )

Оцененное среднее: -0.10384651739409384
Оцененное стандартное отклонение: 0.9036161766446296
Оцененная дисперсия: 0.8165221946938583

Результаты теста:
Статистика теста: 91.3348
Критическое значение (alpha=0.05, df=2): 5.9915
p-value: 0.0000



Отвергаем H0: данные не согласуются с предположением μ = 1.0, sigma^2 = 2.0000000000000004


## Практика

* На практике для давно известных распределений никто не оценивает параметры ММП в каждой задаче, а просто используют уже известные оценки, выведенные в общем виде.

* Например, для аналитики спорта очень часто используют Пуассоновское распределение.

* Им моделируют, например, общее кол-во голов в матче, голы домашней команды, команды соперника, кол-во успешных действий конкретного игрока, кол-во красных и желтых карточек... В общем любой "подсчет" в единицу времени

* В этой практике попробуем немного погрузиться в экономику спорта

В данной задачи будем использовать данные с <https://footystats.org/> о матчах в самых известных футбольных премьер лигах за 5 последних сезонов.

Данные можно загрузить по ссылке: https://drive.google.com/drive/folders/16WZMzHd74W-O_NdUgsEPjJ9rTXywsN1t?usp=drive_link

Описание данных можно найти здесь: https://footystats.org/api/documentations/match-schedule-and-stats

### TASK 1 (OPTIONAL) 

Если есть желание попрактиковаться, попробуйте вывести оценки ММП для распределение Пуассона в общем виде

In [ ]:
### YOUR CODE OR LATEX HERE

### TASK 2: Спорт как Пуассоновская модель

* Объедините все голы, забитые во всех матчах: и домашние, и гостевые, либо возьмите готовый столбец из данных. 

* Предположив, что общее количество голов в лиге подчиняется распределению Пуассона, найдите оценку максимального правдоподобия для параметра λ (среднее количество голов за матч в лиге)

* Насколько она устойчива к изменению выборки (например, если взять только матчи одной команды или одного сезона)?

* Графические иллюстрации приветствуются, но не обязательны :)

In [ ]:
def load_all_leagues_data():
    base_path = "../data/Footystats/"
    all_leagues_data = []

    league_folders = [
        f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))
    ]

    for league_folder in league_folders:
        league_path = os.path.join(base_path, league_folder)
        csv_files = glob.glob(os.path.join(league_path, "*.csv"))

        league_matches = 0
        for file in csv_files:
            try:
                df_league = pd.read_csv(file)

                df_league["league"] = league_folder

                filename = os.path.basename(file)
                season_parts = filename.split("-")
                if len(season_parts) >= 5:
                    season = f"{season_parts[-5]}-{season_parts[-4].split('.')[0]}"
                else:
                    season = "Unknown"
                df_league["season"] = season

                all_leagues_data.append(df_league)
                league_matches += len(df_league)

            except Exception as e:
                print(f"Ошибка загрузки {file}: {e}")

    if all_leagues_data:
        df_all = pd.concat(all_leagues_data, ignore_index=True)
        return df_all


df_all = load_all_leagues_data()

In [ ]:
def convert_footystats_date(date_str):
    try:
        date_part = date_str.split(" - ")[0]
        return datetime.datetime.strptime(date_part, "%b %d %Y")
    except:
        return pd.NaT

In [ ]:
home_goals_col = "home_team_goal_count"
away_goals_col = "away_team_goal_count"

total_goals_all = df_all["total_goal_count"].sum()
total_matches_all = len(df_all)
lambda_mle_all = total_goals_all / total_matches_all

print("1. ОЦЕНКА ММП ДЛЯ ВСЕХ ДАННЫХ:")
print(f"\tВсего матчей: {total_matches_all:,}")
print(f"\tВсего голов: {total_goals_all:,}")
print(f"\tlambda_mle_all = {lambda_mle_all:.4f}")

goal_counts = df_all["total_goal_count"].value_counts().sort_index()
x_range = np.arange(0, min(goal_counts.index.max() + 1, 14))
poisson_pmf = stats.poisson.pmf(x_range, lambda_mle_all) * total_matches_all

observed = goal_counts.reindex(x_range, fill_value=0)
poisson_pmf_normalized = poisson_pmf * (observed.sum() / poisson_pmf.sum())
chi2_stat, p_value = stats.chisquare(observed, poisson_pmf_normalized)

fig1 = go.Figure()

fig1.add_trace(
    go.Bar(
        x=goal_counts.index,
        y=goal_counts.values,
        name="Наблюдаемые данные",
        opacity=0.7,
        marker_color="lightblue",
        text=goal_counts.values,
    )
)
fig1.add_trace(
    go.Scatter(
        x=x_range,
        y=poisson_pmf_normalized,
        mode="lines+markers",
        name=f"Пуассон (lambda={lambda_mle_all:.3f})",
    )
)
fig1.update_layout(
    title=f"Распределение голов: Наблюдаемые данные vs Пуассоновская модель lambda = {lambda_mle_all:.3f}, p-value = {p_value:.4f}",
    xaxis_title="Количество голов в матче",
    yaxis_title="Количество матчей",
)
fig1.show()

1. ОЦЕНКА ММП ДЛЯ ВСЕХ ДАННЫХ:
	Всего матчей: 24,255
	Всего голов: 67,183.0
	lambda_mle_all = 2.7699


In [113]:
season_lambdas = []
for season in df_all["season"].unique():
    season_data = df_all[df_all["season"] == season]
    lambda_season = season_data["total_goal_count"].mean()
    season_lambdas.append(
        {
            "season": season,
            "lambda": lambda_season,
        }
    )

df_seasons = pd.DataFrame(season_lambdas).sort_values("season")

fig2 = go.Figure()
fig2.add_trace(
    go.Scatter(
        x=df_seasons["season"],
        y=df_seasons["lambda"],
        mode="lines+markers+text",
        name="λ по сезонам",
    )
)
fig2.add_hline(
    y=lambda_mle_all,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Общее λ = {lambda_mle_all:.3f}",
)
fig2.update_layout(
    title="Устойчивость оценки λ по сезонам",
    xaxis_title="Сезон",
    yaxis_title="λ (средние голы за матч)",
)
fig2.show()

In [ ]:
league_lambdas = []
for league in df_all["league"].unique():
    league_data = df_all[df_all["league"] == league]
    lambda_league = league_data["total_goal_count"].mean()
    league_lambdas.append(
        {
            "league": league,
            "lambda": lambda_league,
        }
    )

df_leagues = pd.DataFrame(league_lambdas).sort_values("lambda", ascending=False)

fig3 = go.Figure()
fig3.add_trace(
    go.Bar(
        x=df_leagues["league"],
        y=df_leagues["lambda"],
        text=[f"{val:.3f}" for val in df_leagues["lambda"]],
    )
)
fig3.add_hline(
    y=lambda_mle_all,
    line_dash="dash",
    line_color="blue",
    annotation_text=f"Общее λ = {lambda_mle_all:.3f}",
)
fig3.update_layout(
    title="Устойчивость оценки λ по лигам",
    xaxis_title="Лига",
    yaxis_title="λ (средние голы за матч)",
)
fig3.show()

In [122]:
all_teams = pd.concat([df_all["home_team_name"], df_all["away_team_name"]])
team_match_counts = all_teams.value_counts()
top_teams = team_match_counts.index

team_lambdas = []
for team in top_teams:
    team_home = df_all[df_all["home_team_name"] == team]
    team_away = df_all[df_all["away_team_name"] == team]
    team_matches = pd.concat([team_home, team_away])

    total_goals_team = team_matches["total_goal_count"].sum()
    lambda_team = total_goals_team / len(team_matches)

    team_lambdas.append(
        {
            "team": team,
            "lambda": lambda_team,
        }
    )

df_teams = pd.DataFrame(team_lambdas).sort_values("lambda", ascending=False)

fig4 = go.Figure()
fig4.add_trace(
    go.Bar(
        x=df_teams["team"],
        y=df_teams["lambda"],
        text=[f"{val:.3f}" for val in df_teams["lambda"]],
    )
)
fig4.add_hline(
    y=lambda_mle_all,
    line_dash="dash",
    line_color="blue",
    annotation_text=f"Общее λ = {lambda_mle_all:.3f}",
)
fig4.update_layout(
    title="Устойчивость оценки λ по командам (топ-20 по количеству матчей)",
    xaxis_title="Команда",
    yaxis_title="λ (средние голы за матч)",
)
fig4.show()

### TASK 3: Стиль игры

1. Выберите одну конкретную команду. Оцените два параметра:

* λ_attack – среднее количество голов, забиваемое этой командой за матч (используйте данные по всем матчам, где она играла).

* λ_defence – среднее количество голов, пропускаемых этой командой за матч.

2. Сравните λ_attack и λ_defence для этой команды. Что это говорит о ее стиле игры? Является ли она атакующей, оборонительной или сбалансированной?

In [126]:
selected_team = "PSG"
team_home_matches = df_all[df_all['home_team_name'] == "PSG"]
team_away_matches = df_all[df_all['away_team_name'] == "PSG"]

print("PSG")
print(f"Домашних матчей: {len(team_home_matches)}")
print(f"Гостевых матчей: {len(team_away_matches)}")
print(f"Всего матчей: {len(team_home_matches) + len(team_away_matches)}")

PSG
Домашних матчей: 91
Гостевых матчей: 91
Всего матчей: 182


In [ ]:
def calculate_team_metrics(home_matches, away_matches, team_name):
    goals_for_home = home_matches['home_team_goal_count'].sum()
    goals_for_away = away_matches['away_team_goal_count'].sum()
    total_goals_for = goals_for_home + goals_for_away
    
    goals_against_home = home_matches['away_team_goal_count'].sum()
    goals_against_away = away_matches['home_team_goal_count'].sum()
    total_goals_against = goals_against_home + goals_against_away
    
    total_matches = len(home_matches) + len(away_matches)
    
    lambda_attack = total_goals_for / total_matches
    lambda_defense = total_goals_against / total_matches
    
    
    return {
        'team': team_name,
        'total_matches': total_matches,
        'total_goals_for': total_goals_for,
        'total_goals_against': total_goals_against,
        'lambda_attack': lambda_attack,
        'lambda_defense': lambda_defense,
        'goal_difference': total_goals_for - total_goals_against,
        'attack_advantage': lambda_attack - lambda_defense
    }

team_metrics = calculate_team_metrics(team_home_matches, team_away_matches, selected_team)

print(f"Всего забито голов: {team_metrics['total_goals_for']}")
print(f"Всего пропущено голов: {team_metrics['total_goals_against']}")
print(f"Разница голов: {team_metrics['goal_difference']:.0f}")
print(f"λ_attack (средние забитые голы/матч): {team_metrics['lambda_attack']:.4f}")
print(f"λ_defense (средние пропущенные голы/матч): {team_metrics['lambda_defense']:.4f}")
print(f"Разница (атака - оборона): {team_metrics['attack_advantage']:.4f}")

Всего забито голов: 438.0
Всего пропущено голов: 172.0
Разница голов: 266
λ_attack (средние забитые голы/матч): 2.4066
λ_defense (средние пропущенные голы/матч): 0.9451
Разница (атака - оборона): 1.4615


In [136]:
attack_data = pd.concat([
    team_home_matches['home_team_goal_count'], 
    team_away_matches['away_team_goal_count']
])
defense_data = pd.concat([
    team_home_matches['away_team_goal_count'],
    team_away_matches['home_team_goal_count']
])

# t-тест т.к. мало наблюдений
t_stat, p_value = stats.ttest_ind(attack_data, defense_data, equal_var=False)

print(f"t-тест: t = {t_stat:.4f}, p-value = {p_value:.4f}")
if p_value < 0.05:
    print(f"Статистически значимое различие между атакой и обороной")
else:
    print(f"Нет статистически значимого различия")


t-тест: t = 11.1367, p-value = 0.0000
Статистически значимое различие между атакой и обороной


In [151]:
fig_style = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Сравнение атаки и обороны",
        "Распределение голов за матч",
        "Динамика по сезонам",
    ),
)

fig_style.add_trace(
    go.Bar(
        x=["Атака (λ_attack)", "Оборона (λ_defense)"],
        y=[team_metrics["lambda_attack"], team_metrics["lambda_defense"]],
        marker_color=["green", "red"],
        name="Параметры команды",
    ),
    row=1,
    col=1,
)

attack_counts = attack_data.value_counts().sort_index()
fig_style.add_trace(
    go.Bar(
        x=attack_counts.index,
        y=attack_counts.values,
        name="Забитые голы",
        marker_color="green",
        text=attack_counts.values,
    ),
    row=1,
    col=2,
)

defense_counts = defense_data.value_counts().sort_index()
fig_style.add_trace(
    go.Bar(
        x=defense_counts.index,
        y=defense_counts.values,
        name="Пропущенные голы",
        marker_color="red",
        text=defense_counts.values,
    ),
    row=1,
    col=2,
)

season_attack = []
season_defense = []

for season in df_all["season"].unique():
    season_home = team_home_matches[team_home_matches["season"] == season]
    season_away = team_away_matches[team_away_matches["season"] == season]

    season_metrics = calculate_team_metrics(season_home, season_away, selected_team)
    season_attack.append(
        {"season": season, "lambda_attack": season_metrics["lambda_attack"]}
    )
    season_defense.append(
        {"season": season, "lambda_defense": season_metrics["lambda_defense"]}
    )

df_season_attack = pd.DataFrame(season_attack).sort_values("season")
df_season_defense = pd.DataFrame(season_defense).sort_values("season")

fig_style.add_trace(
    go.Scatter(
        x=df_season_attack["season"],
        y=df_season_attack["lambda_attack"],
        mode="lines+markers",
        name="Атака",
        line=dict(
            color="green",
        ),
    ),
    row=2,
    col=1,
)

fig_style.add_trace(
    go.Scatter(
        x=df_season_defense["season"],
        y=df_season_defense["lambda_defense"],
        mode="lines+markers",
        name="Оборона",
        line=dict(
            color="red",
        ),
    ),
    row=2,
    col=1,
)

fig_style.update_layout(
    title=f"Анализ стиля игры: {selected_team}",
)

fig_style.update_yaxes(title_text="λ (голы/матч)", row=1, col=1)
fig_style.update_yaxes(title_text="Количество матчей", row=1, col=2)
fig_style.update_yaxes(title_text="λ (голы/матч)", row=2, col=1)

fig_style.show()

### TASK 4: Существует ли эффект домашнего поля?

* Формулировка гипотезы: Команды в среднем забивают больше голов, играя дома, чем в гостях.


1. Для выбранной команды (или для всех команд в совокупности) оцените λ_home (средние голы дома) и λ_away (средние голы в гостях).

2. Постройте 95% доверительные интервалы для λ_home и λ_away (можно использовать свойство асимптотической нормальности оценки ММП). Перекрываются ли они?

3. Проверьте гипотезу H₀: λ_home = λ_away против H₁: λ_home > λ_away (Hint: можно использовать тест отношения правдоподобия).

In [157]:
home_goals_all = df_all['home_team_goal_count'].sum()
away_goals_all = df_all['away_team_goal_count'].sum()

total_home_matches = len(df_all)
total_away_matches = len(df_all)  

lambda_home = home_goals_all / total_home_matches
lambda_away = away_goals_all / total_away_matches

print(f"λ_home = {lambda_home:.4f}")
print(f"λ_away = {lambda_away:.4f}")


def calculate_poisson_ci(lambda_est, n, alpha=0.05):
    z = stats.norm.ppf(1 - alpha/2)
    se = np.sqrt(lambda_est / n)
    ci_lower = lambda_est - z * se
    ci_upper = lambda_est + z * se
    return ci_lower, ci_upper, se

ci_home_lower, ci_home_upper, se_home = calculate_poisson_ci(lambda_home, total_home_matches)
ci_away_lower, ci_away_upper, se_away = calculate_poisson_ci(lambda_away, total_away_matches)

print(f"λ_home: [{ci_home_lower:.4f}, {ci_home_upper:.4f}]")
print(f"λ_away: [{ci_away_lower:.4f}, {ci_away_upper:.4f}]")
print("Доверительные интервалы не перекрываются")

λ_home = 1.5223
λ_away = 1.2475
λ_home: [1.5068, 1.5379]
λ_away: [1.2335, 1.2616]
Доверительные интервалы не перекрываются


In [159]:
def poisson_log_likelihood(lambda_param, goals_data):
    return np.sum(stats.poisson.logpmf(goals_data, lambda_param))

home_goals_data = df_all['home_team_goal_count'].values
away_goals_data = df_all['away_team_goal_count'].values

# H0
lambda_common = (home_goals_all + away_goals_all) / (total_home_matches + total_away_matches)

# H1
lambda_home_mle = lambda_home
lambda_away_mle = lambda_away

print(f"H0: λ_common = {lambda_common:.4f}")
print(f"H1: λ_home = {lambda_home_mle:.4f}, λ_away = {lambda_away_mle:.4f}")

logL_H0 = (poisson_log_likelihood(lambda_common, home_goals_data) + 
          poisson_log_likelihood(lambda_common, away_goals_data))

logL_H1 = (poisson_log_likelihood(lambda_home_mle, home_goals_data) + 
          poisson_log_likelihood(lambda_away_mle, away_goals_data))

print(f"logL(H0) = {logL_H0:.4f}")
print(f"logL(H1) = {logL_H1:.4f}")

LR_statistic = -2 * (logL_H0 - logL_H1)
print(f"Статистика теста: {LR_statistic:.4f}")

df = 1
critical_value = stats.chi2.ppf(0.95, df) 
p_value = 1 - stats.chi2.cdf(LR_statistic, df)

print(f"Критическое значение (alpha=0.05): chi^2(1) = {critical_value:.4f}")
print(f"p-value: {p_value:.6f}")

if p_value < 0.05:
    print(f"Эффект домашнего поля статистически значим")
else:
    print(f"Эффекта домашнего поля не статистически значим")

H0: λ_common = 1.3849
H1: λ_home = 1.5223, λ_away = 1.2475
logL(H0) = -73708.0854
logL(H1) = -73376.9348
Статистика теста: 662.3012
Критическое значение (alpha=0.05): chi^2(1) = 3.8415
p-value: 0.000000
Эффект домашнего поля статистически значим
